In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.utils.data as data_utils

In [2]:
train_path = '/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/new/'

In [3]:
#input_ = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_input.npy')

In [4]:
#output_ = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_output.npy')

In [5]:
input_dev = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_input_dev.npy')

In [6]:
output_dev = np.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/data/processed/ffnn_output_dev.npy')

In [7]:
list_of_train_files = []
list_of_label_files = []
for path, subdirs, files in os.walk(train_path):
    for name in files:
        if name.startswith("ffnn_train"):
            list_of_train_files.append(os.path.join(path, name))
        if name.startswith("ffnn_labels"):
            list_of_label_files.append(os.path.join(path, name))
list_of_train_files = sorted(list_of_train_files)
list_of_label_files = sorted(list_of_label_files)
        

In [8]:
len(list_of_train_files)

100

In [9]:
len(list_of_label_files)

100

In [10]:
inp = np.load(list_of_train_files[0])

In [11]:
inp[0].shape

(1237, 1)

In [12]:
input_dev[0].shape

(1237, 1)

In [13]:
INPUT_DIM = 1237
HIDDEN_DIM1 = 1000
HIDDEN_DIM2 = 500
HIDDEN_DIM3 = 500
OUTPUT_DIM = 2
BATCH_SIZE = 32
NUM_EPOCHS = 1

In [14]:
def dataLoader(start_index, batch_size, inputs, labels, input_dim):
    '''
    This function provides the data to the network in batches from the starting position start_index.
    Args: 
       start_index (int): Position to start the batch from the dataset.
       batch_size (int): Batch size.
       inputs: The inputs to the model.
       labels: The expected outputs from the model.
    
    Returns: 
        train_tensor(torch tensor): Tensor of batch size.
        label_tensor(torch tensor): Tensor of batch size.
    '''
    
    inputs_tensor_list = []
    labels_tensor_list = []
    
    if len(inputs[start_index:]) < batch_size:
        batch_size = len(inputs[start_index:])
    
    for pos in range(start_index, start_index + batch_size):
        inputs_tensor_list.append((torch.from_numpy(inputs[pos].reshape(input_dim))).float())
        labels_tensor_list.append(torch.from_numpy(labels[pos]))
        
    inputs_tensor = torch.Tensor(batch_size, input_dim)
    labels_tensor = torch.LongTensor(batch_size, 1)
    torch.cat(inputs_tensor_list, out = inputs_tensor)
    torch.cat(labels_tensor_list, out = labels_tensor)
    inputs_tensor = inputs_tensor.view(batch_size, 1237)
    
    return inputs_tensor, labels_tensor

In [20]:
def computeF1(model, dev_input, dev_label):
    '''
    This function computes the F1 score of the model.
    
    Args:
        model: The model which is being used to compute the F1 score.
        inputs: The inputs to the model.
        labels: The expected outputs from the model.
    
    Returns:
        tp(int):        True Positives.
        fp(int):        False Positives.
        fn(int):        False Negaitves.
        precision(int): The Precision of the F1 score.
        recall(int):    The recall of the F1 score. 
        f1(int):        The computed F1 score.
        
    '''
    tp = 0
    fp = 0
    fn = 0
    for j in range(0, len(dev_input), BATCH_SIZE):
        inputs, labels = dataLoader(j, BATCH_SIZE, dev_input, dev_label, INPUT_DIM)
        inputs = Variable(inputs)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        for l, p in zip(labels, predicted):
            if l == 1 and p == 1:
                tp += 1
            if l == 0 and p == 1:
                fp += 1
            if l == 1 and p == 0:
                fn += 1
    
    if tp == 0 and fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)
    if tp == 0 and fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)
    if precision == 0 and recall == 0:
        f1 = 0
    else:
        f1 = (2 * precision * recall) / (precision + recall)
        
    return tp, fp, fn, precision, recall, f1

In [21]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(FFNN, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()

        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()

        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.relu3 = nn.ReLU()

        self.fc4 = nn.Linear(hidden_dim2, output_dim)
    
    def forward(self, x):
        #x = x.view(-1, 1337)
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        
        return out

In [43]:
model = FFNN(INPUT_DIM, HIDDEN_DIM1, HIDDEN_DIM2, HIDDEN_DIM3, OUTPUT_DIM)

In [44]:
LEARNING_RATE = 1e-1
WEIGHT_DECAY = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE)

# Loss Function
criterion = nn.CrossEntropyLoss()

In [46]:
iterations = 0
bestF1 = 0
for epoch in range (1):
    for t, l in zip(list_of_train_files, list_of_label_files):
        input_ = np.load(t)
        output_ = np.load(l)
        for i in range(0, len(input_), BATCH_SIZE):
            inputs, labels = dataLoader(i, BATCH_SIZE, input_, output_, INPUT_DIM)
            inputs , labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = model(inputs)   
            loss = criterion(outputs, labels)        
            loss.backward()
            optimizer.step()
            iterations += 1

            if iterations % 500 == 0:
                tp, fp, fn, precision, recall, f1_ = computeF1(model, input_dev, output_dev)
                print ('Epoch: ' + str(epoch))
                #print ('Iteration: ' + str(iterations))
                #print ('True Positives: ' + str(tp))
                #print ('False Positives: ' + str(fp))
                #print ('False Negatives: ' + str(fn))
                print ('F1: ' + str(f1_))
                if f1_ > bestF1:
                    bestF1 = f1_
                    torch.save(model, '/home/vishesh/TUM/Thesis/Coreference-Resolution/models/bestModel.pt')
                    print ('model saved!')

Epoch: 0
F1: 0.21151057640762452
model saved!


/home/vishesh/anaconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type FFNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 0
F1: 0.5268966040311724
model saved!
Epoch: 0
F1: 0.31060929175746205
Epoch: 0
F1: 0.6665723281565076
model saved!
Epoch: 0
F1: 0.5985920517516112
Epoch: 0
F1: 0.5088947661469932
Epoch: 0
F1: 0.5611987128413225
Epoch: 0
F1: 0.5547043584178862
Epoch: 0
F1: 0.28232068674406746
Epoch: 0
F1: 0.5929797992134639
Epoch: 0
F1: 0.06841376080551057
Epoch: 0
F1: 0.5976347584492497
Epoch: 0
F1: 0.5839078901088656
Epoch: 0
F1: 0.5088513791683821
Epoch: 0
F1: 0.615568233356408
Epoch: 0
F1: 0.5602112115073855
Epoch: 0
F1: 0.6087022721948934
Epoch: 0
F1: 0.6731854636459154
model saved!
Epoch: 0
F1: 0.6248185571552022
Epoch: 0
F1: 0.6106425432422177
Epoch: 0
F1: 0.6592194508852147
Epoch: 0
F1: 0.6765512803143205
model saved!
Epoch: 0
F1: 0.5043907609534224
Epoch: 0
F1: 0.6765930871110155
model saved!
Epoch: 0
F1: 0.6951924612681679
model saved!
Epoch: 0
F1: 0.2608761596602543
Epoch: 0
F1: 0.43623889937638183
Epoch: 0
F1: 0.2534612561244174
Epoch: 0
F1: 0.5885563363123373
Epoch: 0
F1: 0.59733255

In [47]:
loadedModel = torch.load('/home/vishesh/TUM/Thesis/Coreference-Resolution/models/bestModel.pt')

In [48]:
tp, fp, fn, precision, recall, f1 = computeF1(loadedModel, input_dev, output_dev)

In [49]:
f1

0.7315546919761851

## Checking on real leife inputs